## Loading the iris dataset

In [372]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

In [373]:
iris = datasets.load_iris()

In [374]:
X = iris.data[:, :4] 
Y = iris.target 
feature_size = len(iris.feature_names)
print(iris.target_names)
target_size = 1

['setosa' 'versicolor' 'virginica']


## Splitting the dataset intro train and test

In [375]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.20,random_state = 42) 

## Learning with Context

In [376]:
from torch_mas.head import Head
from torch_mas.data import DataBuffer

model = Head(
    feature_size, 
    target_size, 
    R=0.25,
    imprecise_th=0.5,
    bad_th=0.0015,
    alpha=0.5,
    memory_length=3,
    n_epochs=5
)

## Training the agents

In [377]:
import time
dataset = DataBuffer(X_train,y_train)
t = time.time()
model.fit(dataset)
tt = time.time() - t
print(f"Total training time: {tt}s")

print("Number of agents created:", model.agents.n_agents)

Total training time: 0.9960007667541504s
Number of agents created: 98


## Computing performance

In [378]:
print(type(X_test))

<class 'numpy.ndarray'>


In [379]:
import time
import torch
import math
test_feature_size = X_test.size
t = time.time()
y_pred = model.predict(torch.from_numpy(X_test).float()).round()
y_test = torch.from_numpy(y_test).int()
tt = time.time() - t
tps = tt / test_feature_size
print(f"Total Prediction time: {tt}s for 4 samples")
print(f"Prediction time per sample: {tps}s")

differences = torch.ne(y_pred.squeeze(), y_test)
print(differences)
num_differences = differences.sum().item()
        
print("Differences:", num_differences)

Total Prediction time: 0.004055500030517578s for 4 samples
Prediction time per sample: 3.3795833587646484e-05s
tensor([False,  True,  True, False,  True, False,  True,  True,  True, False,
         True, False, False, False, False,  True,  True, False,  True,  True,
        False,  True, False,  True,  True,  True,  True,  True, False, False])
Differences: 17


# Hyperparameter optimisation

In [380]:
dic = {
    "R" : {0,1},
    "bad_th" : {0,1},
    "alpha" : {0,1},
}
init = 0.1
best = math.inf
bestdic = []
for R in range(1,10):
    for b in range(1,10):
        for a in range(1,10):
            model = Head(
            feature_size, 
            target_size, 
            R=R*init,
            imprecise_th=0.5,
            bad_th=b*init,
            alpha=a*init,
            memory_length=3,
            n_epochs=5
            )
            dataset = DataBuffer(X_train,y_train)
            t = time.time()
            model.fit(dataset)
            y_pred = model.predict(torch.from_numpy(X_test).float()).round()
            
            differences = torch.ne(y_pred.squeeze(), y_test)
            num_differences = differences.sum().item()
            #print("Parameters : ",R*init,":",b*init,":",a*init," Score:",num_differences)
            if(num_differences < best):
                bestdic = [{R,b,a}]
                best = num_differences
            elif num_differences == best:
                bestdic.append({R,b,a})
print("Best parameters", bestdic, " with Score", best)

Best parameters [{8, 1}, {8, 1, 2}, {8, 2, 3}, {8, 9, 2}, {8, 3, 5}, {8, 3, 4}, {8, 4}, {8, 9, 4}, {8, 3, 5}, {8, 1, 6}, {8, 3, 6}, {8, 9, 6}, {8, 5}, {8, 6}, {8, 9, 5}, {9, 1, 6}, {9, 2}, {9, 2, 4}, {9, 2}, {9, 3, 1}, {9, 2, 3}, {9, 3, 5}, {9, 3, 7}, {9, 3, 4}, {9, 4, 7}, {9, 3, 5}, {9, 6, 1}, {9, 3, 6}, {9, 6, 7}, {9, 6, 7}, {9, 7}, {8, 9, 3}, {9, 3}, {9, 5}, {8, 9}]  with Score 0
